In [1]:
!pip install bitsandbytes
!pip install faiss-cpu
!pip install accelerate


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install tqdm
!pip install pandas
!pip install numpy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install transformers


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import gc
import time
from tqdm import tqdm
import numpy as np
import pandas as pd

In [4]:
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import faiss
import torch.quantization

In [5]:
compute_dtype = getattr(torch, "float16")

In [6]:
def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'


In [7]:
import json

data = []
with open('dataset.json', 'r', encoding='utf-8') as file:
    for line in file:
        data.append(json.loads(line))

df = pd.DataFrame(data)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
tokenizer = AutoTokenizer.from_pretrained('Salesforce/SFR-Embedding-Mistral')
tokenizer.add_eos_token = True

model = AutoModel.from_pretrained('Salesforce/SFR-Embedding-Mistral')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
qconfig = torch.quantization.get_default_qconfig('fbgemm')
model_quant = torch.quantization.quantize_dynamic(model, qconfig, dtype=torch.qint8)

KeyboardInterrupt: 

In [21]:
model.to(device)

MistralModel(
  (embed_tokens): Embedding(32000, 4096, padding_idx=2)
  (layers): ModuleList(
    (0-31): 32 x MistralDecoderLayer(
      (self_attn): MistralSdpaAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (rotary_emb): MistralRotaryEmbedding()
      )
      (mlp): MistralMLP(
        (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): MistralRMSNorm()
      (post_attention_layernorm): MistralRMSNorm()
    )
  )
  (norm): MistralRMSNorm()
)

In [14]:
all_index = faiss.IndexIDMap(faiss.IndexFlatIP(4096))

In [15]:
torch.cuda.memory_allocated("cuda")

3678504960

In [16]:
torch.cuda.empty_cache()
gc.collect()

1344

In [17]:
df

,imt_id,nm_id,imt_name,subj_name,subj_root_name,nm_colors_names,vendor_code,description,brand_name
0,100005907,165795732,Бейсболка,Бейсболки,Головные уборы,"желтый, мятный, черный",Ес ж/сет,"Универсальный, стильный и современный головной...",FIVE STARS CAP
1,100005907,165798509,Бейсболка,Бейсболки,Головные уборы,"желтый, мятный, черный",Ес желт,"Универсальный, стильный и современный головной...",FIVE STARS CAP
2,100044945,165791148,Фонтан на солнечной батарее,Насосы погружные,Садовая техника,,AH-099-F-902-black,Садовый декоративный беспроводной фонтан на со...,ALVE home
3,100057269,165741256,Шорты для девочки мальчика детские Для новорож...,Шорты для малышей,Одежда для малышей,белый,181001К-ПРИНЦ,"Вы ищете набор из трех шортиков для мальчика, ...",Лантана
4,100057269,165741325,Шорты для девочки мальчика детские Для новорож...,Шорты для малышей,Одежда для малышей,"белый, зеленый, серый",181001К-Я/Р/П,"Вы ищете набор из трех шортиков для мальчика, ...",Лантана
...,...,...,...,...,...,...,...,...,...
197024,9902347,165643307,Шпажки для канапе 12 см 100 шт,Шпажки,Посуда и инвентарь,"черный, черный лакированный никел",цветок_черный,Шпажки для канапе (пики) используются при серв...,Sweet Edelweiss
197025,9988669,165765018,Бриджи,Бриджи,Одежда,лимонный,1Е607К/89,,Домтекс
197026,9988671,165793170,Бриджи,Бриджи,Одежда,бежевый,5Е0128К/44,,Домтекс
197027,9988871,165606160,Серьги серебряные с камнем,Ювелирные серьги,Ювелирные украшения,"серебристый, серебристо-белый",TJE256MS,Винтажные серьги серебро 925 пробы Николь с на...,ЗОЛОТО АЛЕКСАНДРА ВАСИЛЬЕВА


In [18]:
df = df.reset_index(drop=True)

In [24]:
max_length = 4096

In [88]:
res_dict = json.load(open("embeddings.json",'r'))
for step in tqdm(range(len(res_dict.keys()) + 1, len(df), 1)): 
    
    products = df.loc[step:step+1-1, :]

    batch_dict = tokenizer(products['imt_name'].tolist(), max_length=max_length - 1, padding=True, truncation=True, return_tensors="pt")
    outputs = model(**batch_dict)

    embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)
    embeddings = embeddings.detach().numpy()
    embeddings = np.asarray(embeddings).astype('float32')
    #cur_index = faiss.IndexIDMap(faiss.IndexFlatIP(embeddings.shape[1]))
    idx = np.asarray(list(products['nm_id'].tolist()))
    #cur_index.add_with_ids(embeddings, idx)
    res_dict[str(idx[0])] = [float(val) for val in embeddings[0]]
    del embeddings
    gc.collect()
    #faiss.write_index(all_index, '/kaggle/working/evropa_product_names.index')
    #print(f"Index execution time: {(time.time() - start) / 60}")

  0%|          | 1/961 [02:29<39:53:40, 149.60s/it]


KeyboardInterrupt: 

In [86]:
res_dict2 = json.load(open("embeddings.json",'r'))

In [87]:
len(res_dict2.keys())

38

In [84]:
json.dump(res_dict, open("embeddings.json", 'w' ))

In [44]:
res_dict3 = json.load( open("embeddings.json" ))

In [79]:
res_dict2 

{'165795732': [-0.008014514110982418,
  0.009168727323412895,
  0.0021791786421090364,
  0.0026106988079845905,
  0.00432436540722847,
  -0.0023252288810908794,
  -0.020604748278856277,
  0.01521578524261713,
  0.025234442204236984,
  0.00209447811357677,
  0.015187615528702736,
  -0.013313030824065208,
  -0.003517169039696455,
  -0.013102948665618896,
  0.012225491926074028,
  -0.012101994827389717,
  0.025888005271553993,
  -0.003952402155846357,
  -0.0210148673504591,
  -0.0055349236354231834,
  -0.0028757990803569555,
  -0.006663280073553324,
  -0.02012370526790619,
  -0.012217516079545021,
  0.0034210551530122757,
  0.00985847506672144,
  -0.002056735334917903,
  -0.007188693154603243,
  0.020356247201561928,
  0.007962824776768684,
  -0.005310048349201679,
  0.011933039873838425,
  0.006775645539164543,
  -0.007000702433288097,
  -0.025452954694628716,
  -0.017910854890942574,
  -0.010668068192899227,
  -0.011408500373363495,
  -0.007156913168728352,
  0.0020720455795526505,
  -0

In [78]:
for val in res_dict.keys():
    res_dict2[str(df[0:20][df['imt_id'] == int(val)].iloc[0]['nm_id'])] = res_dict[val]

C:\Users\Kotov\AppData\Local\Temp\ipykernel_868\3605631029.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  res_dict2[str(df[0:20][df['imt_id'] == int(val)].iloc[0]['nm_id'])] = res_dict[val]
C:\Users\Kotov\AppData\Local\Temp\ipykernel_868\3605631029.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  res_dict2[str(df[0:20][df['imt_id'] == int(val)].iloc[0]['nm_id'])] = res_dict[val]
C:\Users\Kotov\AppData\Local\Temp\ipykernel_868\3605631029.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  res_dict2[str(df[0:20][df['imt_id'] == int(val)].iloc[0]['nm_id'])] = res_dict[val]
C:\Users\Kotov\AppData\Local\Temp\ipykernel_868\3605631029.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  res_dict2[str(df[0:20][df['imt_id'] == int(val)].iloc[0]['nm_id'])] = res_dict[val]
C:\Users\Kotov\AppData\Local\Temp\ipykernel_868\3605631029.py:2: UserWarning: Boolean Series

In [46]:
del res_dict3

In [ ]:
for k in res_dict.keys()

In [42]:
res_dict

{100005907: [-0.008014514,
  0.009168727,
  0.0021791786,
  0.0026106988,
  0.0043243654,
  -0.0023252289,
  -0.020604748,
  0.015215785,
  0.025234442,
  0.002094478,
  0.0151876155,
  -0.013313031,
  -0.003517169,
  -0.013102949,
  0.012225492,
  -0.012101995,
  0.025888005,
  -0.003952402,
  -0.021014867,
  -0.0055349236,
  -0.002875799,
  -0.00666328,
  -0.020123705,
  -0.012217516,
  0.0034210552,
  0.009858475,
  -0.0020567353,
  -0.007188693,
  0.020356247,
  0.007962825,
  -0.0053100483,
  0.01193304,
  0.0067756455,
  -0.0070007024,
  -0.025452955,
  -0.017910855,
  -0.010668068,
  -0.0114085,
  -0.007156913,
  0.0020720456,
  -0.0022462518,
  0.0047773323,
  -0.00047131392,
  0.0009451829,
  -0.0022723984,
  -0.002115155,
  0.018847926,
  0.0101443855,
  -0.0021585838,
  0.002148533,
  -0.011793223,
  0.014570878,
  -0.019852884,
  0.27041385,
  -0.0014132124,
  -0.019629648,
  0.010635984,
  0.009584034,
  -0.011873812,
  -0.011153473,
  -0.0017199836,
  -0.001406231,
  0.00

In [40]:
res_dict2 = {}
for key in res_dict.keys():
    res_dict2[str(key)] = [float(val) for val in res_dict[key]]

In [41]:
res_dict2

{'100005907': [-0.008014514110982418,
  0.009168727323412895,
  0.0021791786421090364,
  0.0026106988079845905,
  0.00432436540722847,
  -0.0023252288810908794,
  -0.020604748278856277,
  0.01521578524261713,
  0.025234442204236984,
  0.00209447811357677,
  0.015187615528702736,
  -0.013313030824065208,
  -0.003517169039696455,
  -0.013102948665618896,
  0.012225491926074028,
  -0.012101994827389717,
  0.025888005271553993,
  -0.003952402155846357,
  -0.0210148673504591,
  -0.0055349236354231834,
  -0.0028757990803569555,
  -0.006663280073553324,
  -0.02012370526790619,
  -0.012217516079545021,
  0.0034210551530122757,
  0.00985847506672144,
  -0.002056735334917903,
  -0.007188693154603243,
  0.020356247201561928,
  0.007962824776768684,
  -0.005310048349201679,
  0.011933039873838425,
  0.006775645539164543,
  -0.007000702433288097,
  -0.025452954694628716,
  -0.017910854890942574,
  -0.010668068192899227,
  -0.011408500373363495,
  -0.007156913168728352,
  0.0020720455795526505,
  -0

In [ ]:
faiss.write_index(all_index, '/kaggle/working/evro_names_eng_10000_6.index')

In [ ]:
all_index.ntotal

In [ ]:
def fetch_product_info(dataframe_idx):
    info = df[df['id'].isin([dataframe_idx])].reset_index(drop=True).iloc[0, :]
    #info = df.loc[df['id'] == dataframe_idx, :].reset_index(drop=True).iloc[0, :]
    meta_dict = dict()
    meta_dict['id'] = info['id']
    meta_dict['name'] = info['name']
    #meta_dict['description'] = info['description'][:500]
    return meta_dict

In [ ]:
full_index = faiss.read_index('/kaggle/input/evro-full-index-eng-title/full_index.index')

In [ ]:
task = 'Given a food product search query, retrieve relevant passages that answer the query'

In [ ]:
print(full_index.ntotal)

In [ ]:
def get_embedding(query: str):
    batch_dict = tokenizer([query], max_length=max_length - 1, padding=True, truncation=True, return_tensors="pt")
    outputs = model(**batch_dict)

    embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)
    embeddings = embeddings.detach().numpy()
    embeddings = np.asarray(embeddings).astype('float32')
    print(embeddings[0])
    return embeddings[0]

In [ ]:
def find_products(scores: list[float]) -> int:
    if len(scores) > 1:
        delta = 0
        slice_index = -1
        for i in range(1, len(scores) - 1):
            cur_delta = scores[i] - scores[i-1]
            if cur_delta > delta:
                delta = cur_delta
                slice_index = i
            if cur_delta > 1:
                break
        return scores[:slice_index]
    else:
        return score
                
    

In [ ]:
def search(simple_query, query, top_k, index):
    t = time.time()
    query_vector = get_embedding(query)
    print(query)

    top_k = index.search(np.asarray([query_vector]), top_k)
    scores = top_k[0][0]
    scores = list(filter(lambda x: x >= 0.65, scores))
    #print(scores)
    #scores = find_products(scores)
    
    #print(top_k[0][:20])
    #print('Results in Total Time: {}'.format(time.time() - t))
    top_k_ids = top_k[1].tolist()[0][:len(scores)]
    top_k_ids = np.asarray(top_k_ids)
    results = [fetch_product_info(idx) for idx in top_k_ids]
    for i in range(len(scores)):
        results[i]['product'] = results[i]['name']
        results[i]['translate_search'] = simple_query
        results[i]['score'] = scores[i]
        del results[i]['name']
        del results[i]['id']
    return pd.DataFrame.from_records(results[:25])

In [ ]:
df = pd.read_csv('/kaggle/input/evropa-catalog/catalog_evropa.csv')

In [ ]:

simple_query = row['translate_search']
query = get_detailed_instruct(task, simple_query)
result = search(simple_query, query, top_k=100, index=full_index)
res_test.append(result)
# print(query)
# print("\n")
# for result in results:
#     print('\t', result)
results

In [ ]:
res_test = pd.concat(res_test)

In [ ]:
res_test.to_csv('test_result.csv')

In [ ]:
res_test